<a href="https://colab.research.google.com/github/aldjia95/4dvop-app/blob/master/Projet_Final_R%C3%A9cup%C3%A9rer_les_path_des_fichiers_images_et_effectuer_le_pr%C3%A9processing_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des librairies et accès au google drive


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt 

tf.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Dezipper le dossier

In [ ]:
zip_file = tf.keras.utils.get_file("raw_image.zip", 
                                   "/content/drive/MyDrive/raw_image.zip",
                                   cache_subdir="/content/drive/MyDrive/", archive_format = 'zip',
                                   extract=True)

## Les données

### Récupération des paths de mes fichiers contenant mes images

In [ ]:
import pathlib

#test folder
test_folder = pathlib.Path("/content/drive/MyDrive/testing/")

#train foler croped images
train_folder = pathlib.Path("/content/drive/MyDrive/training")

#### Récupération des chemins de chaque image (en format str)

In [ ]:
# test --> Je transforme également mes images au format jpg.
all_image_paths_for_test = [str(img_path) for img_path in list(test_folder.glob("*.jpg"))]

#train --> Je transforme également mes images au format jpg.
all_image_paths_for_train = [str(img_path) for img_path in list(train_folder.glob("*.jpg"))]


display(all_image_paths_for_test[-5:])
print('\n')
display(all_image_paths_for_train[-5:])

### Déterminer le nombre d'image de mes fichiers à l'aide du nombre des paths

In [ ]:
print('Number of images in Train :')
print(len(all_image_paths_for_train))
print('________________________________________________')
print('Number of images in Test :')
print(len(all_image_paths_for_test))

# Creation des labels 

Nous voulons ici récupérer les labels à partir des titres des images. Nous avons un exemple : ---> '9_img_1553497170231.jpg'

In [ ]:
get_fichier = lambda f: os.path.split(f)[-1]
get_labels =lambda f: [f[0]]


process = lambda f: get_labels(get_fichier(f))

In [ ]:
import os

label_test=[]
for paths in all_image_paths_for_test:
    a = process(paths)
    label_test += a

label_train=[]
for paths in all_image_paths_for_train:
    a = process(paths)
    label_train += a

In [ ]:


label_test = [int(x) for x in label_test]

label_train = [int(x) for x in label_train]



### création de nos variables de validation

* Train = 45000 images

* Test = 5000 images

* Val = 5000 images

In [ ]:
# Récupération des paths et des labels d'entraînement que j'associe dans X et Y : 
X = all_image_paths_for_train
Y = label_train

# Récupération des paths et des labels de test que j'associe dans X_test et Y_test : 
X_test = all_image_paths_for_test
Y_test = label_test

In [ ]:
# Je récupère 10% des paths de train que j'implémente dans X_val et Y_val : 
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=1,shuffle=True) # = 5000 images

In [ ]:
len(X_train)
X_train

### Creation de mes tenseurs 

In [ ]:
tensortrain_set = tf.data.Dataset.from_tensor_slices((X_train))
tensortval_set = tf.data.Dataset.from_tensor_slices((X_val))
tensortest_set = tf.data.Dataset.from_tensor_slices((X_test))

Y_train = tf.data.Dataset.from_tensor_slices(Y_train)
Y_val = tf.data.Dataset.from_tensor_slices(Y_val)  
Y_test= tf.data.Dataset.from_tensor_slices(Y_test)

### Fonction pour transformer mes paths en image et data-augmentation

In [ ]:
def data_aug(img, label):

  img= tf.io.read_file(img) # Lire mes paths en image
  img = tf.image.decode_jpeg(img, channels=3) # Transformer mes images au format jpeg
  img = tf.image.resize(img, [64, 64]) # Mettre mes images à la même échelle
  img = tf.image.random_flip_left_right(img) # Rotation de mes images
  img = tf.image.random_contrast(img, 0.9, 1.2) # Random contrast 
  img = tf.image.random_crop(img, [64,64,3]) # Zoom random des images
  img = img / 255 # Normalisation de mes pixels

  return img, label

In [ ]:
tensortrain_set = tf.data.Dataset.zip((tensortrain_set, Y_train))
train_set = tensortrain_set.repeat(3) # Je multiplie par 3 mes images d'entraînement --> Data-augmentation
train_set = train_set.map(data_aug) # J'applique la fonction data_aug dans mes données d'entrainement

### Transformation des données de test

In [ ]:
def load_and_preprocess_images(img):
  img = tf.io.read_file(img)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [64, 64])
  img = img / 255
  return img

In [ ]:
tensortval_set = tensortval_set.map(load_and_preprocess_images) # J'applique la fonction load_and_preprocess_images dans mes données de validation
val_set = tf.data.Dataset.zip((tensortval_set, Y_val))

tensortest_set = tensortest_set.map(load_and_preprocess_images) # J'applique la fonction load_and_preprocess_images dans mes données de de test
test_set = tf.data.Dataset.zip((tensortest_set, Y_test))

In [ ]:
print('size of repeated_train_set is {} total images'.format(len(train_set)))
print('size of val_set is {} images total'.format(len(val_set)))
print('size of test_set is {} images total'.format(len(test_set)))

In [ ]:
train_set = train_set.shuffle(len(train_set)).batch(10)

val_set = val_set.batch(10)

test_set = test_set.batch(10)

In [ ]:
for example_x, example_y in train_set.take(1):
  plt.figure()
  plt.title(example_y[0].numpy())
  plt.imshow(example_x[0].numpy())
plt.show()

In [ ]:
for x in Y_train:
  print(y[0])

In [ ]:
for x,y in train_set.take(1):
  print(y[0])

In [ ]:
for example_x, example_y in train_set.take(1):
  plt.figure()
  plt.title(example_y[0].numpy())
  plt.imshow(example_x[0].numpy())
plt.show()

## 3 - Deep learning

Now, it's time to build your neural network.
* Create a CNN model in which you will put 2D convolutional layers and MaxPool2D layers.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", input_shape=[60, 60, 3]),
    tf.keras.layers.MaxPool2D(pool_size=21, strides=2, padding='valid'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=10, strides=2, padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(units=32, activation ="relu"),
    tf.keras.layers.Dense(units=16, activation ="relu"),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

* Bonus : Create a _Learning Rate Schedule_. You can choose to do this with [_ExponentialDecay_](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) (with learning_rate = 0.0005)

In [ ]:
#  Let's create a learning rate schedule to decrease the learning rate as we train the model. 
initial_learning_rate = 0.0005

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=6000,
    decay_rate=0.95,
    staircase=True)

* Create a compiler in which you choose : 
  * Your optimizer (with learning_rate = 0.0005)
  * Your loss 
  * Your metrics 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

* Put the model in a variable called "history" and run on 10 epochs. Use `validation_data`argument for your validation data. 

In [ ]:
history = model.fit(X_train, validation_data = X_test, epochs=10)

* Evaluate the model with test data.

What is the conclusion about these results ?

In [ ]:
model.evaluate(ds_test)

* Now watch a graph of how your *loss* evolves as epochs advance.
  * For that, you can use the evolution of your model with `.history` function with "loss", "val_loss" for loss function and "sparse_categorical_accuracy", "val_sparse_categorical_accuracy" for evolution of accuracy. 

In [ ]:
plt.plot(history.history["loss"], color="b")
plt.plot(history.history["val_loss"], color="r")
plt.ylabel("loss")
plt.xlabel("Epochs")
plt.show()

In [ ]:
plt.plot(history.history["sparse_categorical_accuracy"], color="b")
plt.plot(history.history["val_sparse_categorical_accuracy"], color="r")
plt.ylabel("sparse_categorical_accuracy")
plt.xlabel("Epochs")
plt.show()

* Make a **confusion matrix** that determines the predictions between each class (True positive, False positive...)

In [ ]:
predict = model.predict(ds_test)

In [ ]:
import seaborn as sns
figure = plt.figure(figsize=(10, 8))
sns.heatmap(tf.math.confusion_matrix(Y_test,tf.argmax(predict, axis=1).numpy()), annot=True, cmap=plt.cm.Reds)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

What's your conclusion ?

## 4 - Save and load your model

* Save your model in drive. For that, import and use the class [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint).

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("/content/Dossier modèle/Mon_modèle_projet_mnist.h5",
                             monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
history = model.fit(ds_train, validation_data = ds_valid, epochs=10, callbacks=[checkpoint])

* Load your model in your code with [tf.keras.models.load_model](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model).

In [ ]:
model = tf.keras.models.load_model("/content/Dossier modèle/Mon_modèle_projet_mnist.h5")

## What's your conclusion ?